---
<p align="center">
  <img src="https://github.com/lacamposm/course-fundamentals-llms-openai-langchain/raw/main/images/image_igac.jpg" alt="Imagen_IGAC" width="280">
</p>

---

# ***Fundamentos de LLMs con Python: Explorando ChatGPT y LangChain***

---

#### ***Instructor: [Luis Andrés Campos Maldonado](https://www.linkedin.com/in/lacamposm/)***

##### ***Email: luisandres.campos@igac.gov.co***

##### ***Contratista-Observatorio Inmobiliario Catastral***





---

# ***Clase 04 - 15 de Marzo de 2024***
---

## ***El Concepto de LLM-chain.***  

**Objetivos de Aprendizaje:**

- Introducir el concepto de LLM-chain y su importancia en el desarrollo avanzado de aplicaciones de IA.
- Explorar cómo se pueden combinar diferentes modelos de lenguaje para resolver tareas complejas.

## ***LLMs y ChatModels***

El modelo de lenguaje es el core de cualquier aplicación o funcionalidad para resolver tareas de NLP con LangChain. Esta libreria nos ofrece 2 sabores:


### ***LLMs***

Son sistemas de AI entrenados en extensos conjuntos de datos de texto, capaces de comprender y generar lenguaje humano de manera coherente completando el texto recibido como input, también llamado prompt. El prompt es una cadena de texto que usualmente se utiliza para dirigir o dar contexto al modelo, indicándole qué tipo de respuesta o continuación generar.

###  ***ChatModels***

ChatModels, aunque a menudo se basan en LLMs, están "afinados" específicamente para mantener conversaciones. A diferencia de los LLMs puros, los ChatModels utilizan una interfaz diferente. En lugar de un único prompt en forma de cadena, toman una lista de mensajes de chat como entrada y devuelven un mensaje de respuesta de la AI como salida.


#### ***Nota:***

Los diferentes tipos de modelos tienen esquemas de entrada y salida distintos, lo que significa que la forma óptima de interactuar con cada uno puede variar significativamente. Aunque LangChain permite tratar a los LLMs y a los ChatModels de manera intercambiable, la estrategia de prompting eficaz puede diferir entre ellos, y los prompts deben ser diseñados específicamente para el tipo de modelo que se esté utilizando.

## ***Chains***

El concepto de Chains en LangChain se basa en la capacidad de secuenciar llamadas a diferentes servicios o pasos de procesamiento de forma que puedan encadenarse para ejecutar procesos más complejos de manera eficiente, con la ventaja de modificar fácilmente la secuencia y beneficiarse de características como la ejecución asíncrona y la observabilidad.

### ***PromptTemplates***

Los prompt templates o plantillas de prompts son recetas predefinidas para generar prompts para modelos de lenguaje. Estas plantillas pueden incluir instrucciones, ejemplos de few-shot learning (aprendizaje con pocos ejemplos), y contexto y preguntas específicos apropiados para una tarea dada.

La biblioteca LangChain proporciona herramientas para crear y trabajar con estas plantillas de prompts. El objetivo es crear plantillas que sean independientes del modelo, lo que significa que se pueden reutilizar a través de diferentes modelos de lenguaje sin necesidad de realizar ajustes significativos para cada modelo. Las plantillas de prompts de LangChain están diseñadas para acomodar estos formatos y facilitar la generación de prompts efectivos que orienten al modelo a realizar la tarea deseada de la manera más eficiente posible.

In [1]:
!pip install pandas openpyxl langchain openai langchain-openai

### ***Ejemplo***

Vamos a crear un LLM-Chain, haciendo uso de los elementos de la libreria.

In [2]:
import json
import pandas as pd
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

OPENAI_API_KEY = "<YOUR_API_KEY>"

In [3]:
## Ejemplo prompt_template

prompt_template = """Cuéntame un chiste {adjetivo} sobre {contenido}""" 
prompt = PromptTemplate.from_template(prompt_template)

prompt.format(adjetivo="elegante", contenido="cocinas")

'Cuéntame un chiste elegante sobre cocinas'

In [4]:
## Observemos que como logramos llevar nuestro prompt a ser dinámico.

prompt.format(adjetivo="largo", contenido="Bogota-Colombia")

'Cuéntame un chiste largo sobre Bogota-Colombia'

In [5]:
## Instanciamos un ChatOpenAI
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo-0125",
    openai_api_key=OPENAI_API_KEY,
    max_tokens=512
)

llm_answer = llm.invoke("Hola. Dame un corto resumen sobre la libreria pandas de python.")
llm_answer

AIMessage(content='Pandas es una biblioteca de código abierto de Python que proporciona estructuras de datos de alto rendimiento y fáciles de usar, así como herramientas de análisis de datos. Permite la manipulación y limpieza de datos, así como el análisis y la visualización de datos de manera sencilla. Con Pandas, los usuarios pueden trabajar con datos tabulares, series temporales y datos de series temporales. Es una herramienta muy popular entre los científicos de datos y los analistas de datos debido a su facilidad de uso y su capacidad para manejar grandes conjuntos de datos.')

In [6]:
## Creamos un LLM-Chain y lo usamos.

llm_chain = LLMChain(llm=llm, prompt=prompt)
llm_chain.invoke(
    {
        "adjetivo": "elegante",
        "contenido": "Caballos" 
    }
)

{'adjetivo': 'elegante',
 'contenido': 'Caballos',
 'text': '¿Por qué los caballos son tan elegantes? Porque siempre van de corbata.'}

## ***Automatizando procesos***

Vamos a considerar una tabla de excel que contiene cuentos cortos sobre diferentes temáticas. Deseamos construir un artefacto que nos proprocione un tema del cuento (sobre una posibles lista de opciones), un título sugerido del mismo, además deseamos que la respuesta este formateada en un JSON.

In [7]:
## Elementos para crear el artefacto.

prompt_template = """Considerando que eres un experto en analizar cuentos, tu tarea es decidir, basándote exclusivamente en la siguiente
lista de opciones: {lista_opciones}, cuál es la mejor etiqueta que describe el cuento proporcionado, NO puedes usar la etiqueta serenidad.
Además, se te pide que asignes un título adecuado para el cuento. Ten en cuenta que cada cuento viene identificado por un ID único, en
este caso es: {id}.

A continuación, se presenta el cuento: {cuento}

Con base en tu análisis, deberás proporcionar tu evaluación utilizando el formato JSON a continuación, asegurándote de rellenar los
campos con el ID del cuento, la etiqueta seleccionada de tu lista de opciones, y el título que sugieres para el cuento.
La estructura de tu respuesta debe ser como se muestra:

[
    "id": "El ID del cuento",
    "etiqueta_asignada": "La etiqueta que asignarás de la lista de opciones",
    "titulo_sugerido": "El título que sugieres para el cuento"
]
"""

prompt = PromptTemplate.from_template(prompt_template)

llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo-0125",
    openai_api_key=OPENAI_API_KEY,
    max_tokens=512
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [8]:
# Data con cuentos.
path_data = "https://raw.githubusercontent.com/lacamposm/course-fundamentals-llms-openai-langchain/main/data/stories.xlsx"
df_stories = pd.read_excel(path_data)
df_stories

,id,tema,cuento
0,c1faabc,amor,"Bajo el farol antiguo, Julieta esperaba con do..."
1,c2beabc,amor,"En las cartas olvidadas en el ático, Esteban e..."
2,c3dcabc,odio,"Entre las dos familias del pueblo, una disputa..."
3,c4efabc,odio,El ajedrez en el parque había sido siempre un ...
4,c5ababc,amistad,Lucía encontró la perrita abandonada en una ca...
5,c6cdabc,amistad,"En el taller de carpintería, dos amigos compet..."
6,c7deabc,familia,"Cada domingo, en la casa de campo, se encendía..."
7,c8efabc,familia,Cuando el pequeño Martín rompió su juguete fav...
8,c9fdabc,rabia,La tempestad interna de Sergio lo llevó a desa...
9,ca0babc,rabia,"En el estruendo de la fábrica, la frustración ..."


In [9]:
list_topic_stories = list(df_stories["tema"].unique())
id = df_stories.loc[3].loc["id"]
cuento = df_stories.loc[3].loc["cuento"]

print("Opciones de tematica:", list_topic_stories)
print("id del cuento:", id)
print("Texto del cuento:", cuento)

Opciones de tematica: ['amor', 'odio', 'amistad', 'familia', 'rabia']
id del cuento: c4efabc
Texto del cuento: El ajedrez en el parque había sido siempre un campo de batalla silencioso entre Edgar y Samuel. Pero cuando el invierno azotó la ciudad y Edgar ya no apareció, Samuel se dio cuenta de que la silla vacía frente a él pesaba más que la victoria más dulce.


In [10]:
answer = llm_chain.invoke(
    {
        "lista_opciones": list_topic_stories,
        "id": id,
        "cuento": cuento
    }
)

answer

{'lista_opciones': ['amor', 'odio', 'amistad', 'familia', 'rabia'],
 'id': 'c4efabc',
 'cuento': 'El ajedrez en el parque había sido siempre un campo de batalla silencioso entre Edgar y Samuel. Pero cuando el invierno azotó la ciudad y Edgar ya no apareció, Samuel se dio cuenta de que la silla vacía frente a él pesaba más que la victoria más dulce.',
 'text': '[\n    {\n        "id": "c4efabc",\n        "etiqueta_asignada": "amistad",\n        "titulo_sugerido": "La silla vacía"\n    }\n]'}

In [11]:
## Respuesta ChatOpenAI
answer["text"]

'[\n    {\n        "id": "c4efabc",\n        "etiqueta_asignada": "amistad",\n        "titulo_sugerido": "La silla vacía"\n    }\n]'

In [12]:
df_stories["lista_opciones"] = [list_topic_stories]*len(df_stories)
df_sugerencia = pd.DataFrame()

for options_format in df_stories.to_dict(orient='records'):
    try:
        json_dict = json.loads(llm_chain.invoke(options_format)["text"])
        df_sugerencia = pd.concat([df_sugerencia, pd.DataFrame(json_dict)], ignore_index=True)
    except Exception as e:
        print(f"Error inesperado: {str(e)}")

df_sugerencia

,id,etiqueta_asignada,titulo_sugerido
0,c1faabc,amor,Bajo el farol antiguo
1,c2beabc,amor,Cartas de Amor en el Ático
2,c3dcabc,amistad,Unión en tiempos de crisis
3,c4efabc,amistad,La silla vacía
4,c5ababc,amistad,El refugio de la perrita
5,c6cdabc,amistad,La obra maestra de la amistad
6,c7deabc,familia,La mesa de los lazos
7,c8efabc,amistad,El valor de la amistad
8,c9fdabc,rabia,La furia interior de Sergio
9,ca0babc,amistad,El silencio compartido


In [13]:
df_final = df_stories.drop(columns=["lista_opciones"]).merge(df_sugerencia, on=["id"], how="inner")
df_final

,id,tema,cuento,etiqueta_asignada,titulo_sugerido
0,c1faabc,amor,"Bajo el farol antiguo, Julieta esperaba con do...",amor,Bajo el farol antiguo
1,c2beabc,amor,"En las cartas olvidadas en el ático, Esteban e...",amor,Cartas de Amor en el Ático
2,c3dcabc,odio,"Entre las dos familias del pueblo, una disputa...",amistad,Unión en tiempos de crisis
3,c4efabc,odio,El ajedrez en el parque había sido siempre un ...,amistad,La silla vacía
4,c5ababc,amistad,Lucía encontró la perrita abandonada en una ca...,amistad,El refugio de la perrita
5,c6cdabc,amistad,"En el taller de carpintería, dos amigos compet...",amistad,La obra maestra de la amistad
6,c7deabc,familia,"Cada domingo, en la casa de campo, se encendía...",familia,La mesa de los lazos
7,c8efabc,familia,Cuando el pequeño Martín rompió su juguete fav...,amistad,El valor de la amistad
8,c9fdabc,rabia,La tempestad interna de Sergio lo llevó a desa...,rabia,La furia interior de Sergio
9,ca0babc,rabia,"En el estruendo de la fábrica, la frustración ...",amistad,El silencio compartido


In [14]:
## Errores fuera de la diagonal principal.
pd.crosstab(df_final["tema"], df_final["etiqueta_asignada"])

etiqueta_asignada,amistad,amor,familia,odio,rabia
tema,,,,,
amistad,12,0,0,0,0
amor,0,12,0,0,0
familia,1,0,7,0,0
odio,7,0,0,5,0
rabia,1,1,1,0,5


## ***Ejercicio***

Debes crear una función con el flujo anterior. Además, busca llevar el ejemplo anterior a un uso de tu contextualizado a tu labol diaria.